<a href="https://colab.research.google.com/github/Abhinaytechie/Excerl/blob/main/6feb_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
dataset=pd.read_csv("/content/sample_data/hate_speech.csv")
dataset.head() # Specifies first 5 rows of datset defautly!

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [52]:
dataset.shape #number of rows AND columns in the dataset

(5242, 3)

In [53]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [54]:
import re
def clean_text(text):
    text=re.sub(r'[^a-zA-Z\']',' ',text)#Filtering alphabets only !
    text=re.sub(r'^\x00-\x7f',' ',text)#Filtering Uicode Characters!
    text=text.lower()#Converting text to lower Case!
    return text

In [55]:
dataset['clean_text']=dataset.tweet.apply(lambda x:clean_text(x))

In [56]:
dataset.head(5)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation


Feature Engineering

In [57]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words=stopwords.words('english')
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [58]:
stop=stopwords.words('english')
#dataset['clean_text']=dataset['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [59]:
dataset.head(6)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...


In [60]:
def gen_freq(text):
  word_list=[]
  for word in text.split():
    word_list.extend(word)
  word_freq=pd.Series(word_list).value_counts()
  word_freq=word_freq.drop(stop_words,errors='ignore')
  return word_freq

In [61]:
def any_neg(words):
  for word in words:
    if word in ['n','no','not','non'] or re.search:
      return 1
    else:
      return 0

In [62]:
def any_rare(words,rare_100):
   for word in words:
     if word in rare_100:
       return 1
     else:
       return 0

In [63]:
def is_question(words):
  for word in words:
    if word in ['when','what','how','why','who']:
      return 1
    else:
      return 0

In [64]:
word_freq=gen_freq(dataset.clean_text.str)
rare_100=word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x:len(x))
dataset['char_count'] = dataset['clean_text'].apply(lambda x: len(str(x)))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x:any_neg(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x:any_rare(x,rare_100))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x:is_question(x))
dataset.head(5)

,id,label,tweet,clean_text,word_count,char_count,any_neg,any_rare,is_question
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...,18,102,1,0,0
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...,19,122,1,0,0
2,3,0,bihday your majesty,bihday your majesty,3,21,1,0,0
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...,12,86,1,0,0
4,5,0,factsguide: society now #motivation,factsguide society now motivation,4,39,1,0,0


In [65]:
from tkinter import Y
from sklearn.model_selection import train_test_split
X=dataset[['word_count','any_neg','any_rare','char_count','is_question']]
Y=dataset.label
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)


In [66]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model=model.fit(X_train,Y_train)
pred=model.predict(X_test)
#

In [67]:
model.predict(X_test[5:10])

array([0, 1, 1, 1, 1])

In [68]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(Y_test,pred)*100,"%")

Accuracy:  42.61201143946616 %


In [69]:
from sklearn.ensemble import RandomForestClassifier
RFmodel=RandomForestClassifier()
RFmodel.fit(X_train,Y_train)
pred=RFmodel.predict(X_test).astype(int)
print("Accuracy: ",accuracy_score(Y_test,pred)*100,"%")

Accuracy:  59.389895138226876 %


In [77]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(f"Accuracy: {accuracy_score(Y_test,pred)*100:.4f}%")
print()
print("Confusion Matrxix:\n",confusion_matrix(Y_test,pred))
print()
print("Classification Report:\n",classification_report(Y_test,pred))

Accuracy: 59.3899%

Confusion Matrxix:
 [[407 192]
 [234 216]]

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.68      0.66       599
           1       0.53      0.48      0.50       450

    accuracy                           0.59      1049
   macro avg       0.58      0.58      0.58      1049
weighted avg       0.59      0.59      0.59      1049

